In [1]:
%matplotlib inline

import io, os, sys, types, pickle

import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator

import numpy as np
from numpy.linalg import eig, inv

import math

from scipy import interpolate, spatial, stats

import seaborn as sns

import skimage.io as skiIo
from skimage import exposure, img_as_float, filters, morphology, transform

from sklearn import linear_model
from sklearn import metrics

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/lily/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-3149eb814adf>", line 1, in <module>
    get_ipython().run_line_magic('matplotlib', 'inline')
  File "/Users/lily/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2313, in run_line_magic
    result = fn(*args, **kwargs)
  File "</Users/lily/anaconda/lib/python3.6/site-packages/decorator.py:decorator-gen-108>", line 2, in matplotlib
  File "/Users/lily/anaconda/lib/python3.6/site-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/Users/lily/anaconda/lib/python3.6/site-packages/IPython/core/magics/pylab.py", line 99, in matplotlib
    gui, backend = self.shell.enable_matplotlib(args.gui.lower())
  File "/Users/lily/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py

FileNotFoundError: [Errno 2] No such file or directory

In [2]:
""" ============== path settings =============="""

' ============== path settings =============='

In [3]:
global isMac
isMac = True

if(isMac):
#     dataPrefix = '/Users/lily/Lily/Academic/AW_Lab/data/fate_switching_gfp_rfp_old/Data'
#     figOutPrefix = '/Users/lily/Lily/Academic/AW_Lab/data/fate_switching_gfp_rfp_old/Output/FigureOutput'
#     dataOutPrefix = '/Users/lily/Lily/Academic/AW_Lab/data/fate_switching_gfp_rfp_old/Output/DataOutput'
    dataPrefix = '/Users/lily/Lily/Academic/AW_Lab/data/fate_switching_gal80/Summary/Data'
    figOutPrefix = '/Users/lily/Lily/Academic/AW_Lab/data/fate_switching_gal80/Summary/FigureOutput1'
    dataOutPrefix = '/Users/lily/Lily/Academic/AW_Lab/data/fate_switching_gal80/Summary/DataOutput1'
else:
#     sys.path.insert(0, 'Z:\\lab\\Projects\\NSP\\NSP_Code\\python\\Quantification_fromPositionOnly\\Functions')
    dataPrefix = 'Z:\\lab\\Projects\\NSP\\Data_Analysis\\Experiments\\Fate_Switching\\GFP_RFP_Gal80\\Summaries\\Data'
    figOutPrefix = 'Z:\\lab\\Projects\\NSP\\Data_Analysis\\Experiments\\Fate_Switching\\GFP_RFP_Gal80\\Summaries\\FigureOutput'
    dataOutPrefix = 'Z:\\lab\\Projects\\NSP\\Data_Analysis\\Experiments\\Fate_Switching\\GFP_RFP_Gal80\\Summaries\\DataOutput'
#     dataPrefix = 'D:\\WEIYUE\\Data_Analysis\\Summary\\Data'
#     figOutPrefix = ''
#     dataOutPrefix = 'D:\\WEIYUE\\Data_Analysis\\Summary\\DataOutput'

In [4]:
### file parameters
# file folders
imageFolder = 'Images'
ROIFolder = 'ROIs'
summaryFolder = 'Annotations'
dataFolder = 'DataOutput'

# file name
summaryName = 'Fz_Gal80_s1r1_summary.csv'

In [5]:
""" ============== Golbal variables and parameters =============="""

' ============== Golbal variables and parameters =============='

In [6]:
### global variables
global ColorCode, targetIndexMatch, bins, channel_mapping
bins = 4096

In [7]:
### analysis parameters
radiusExpanseRatio = [2.5, 3]
# num_angleSection = 20
# num_outsideAngle = 20
num_angleSection = 10 # must be even number
num_outsideAngle = 2
num_Xsection = 10
z_offset = 5

analysisParams = (num_angleSection, num_outsideAngle, num_Xsection, z_offset, radiusExpanseRatio)


In [8]:
### import custom functions
# sys.path.insert(0, '/Volumns/wji/lab/Projects/NSP/NSP_Code/python/Quantification_fromPositionOnly/Functions/')
import Data_quantification_function_helper as my_help
import Data_quantification_function_intensity_calculation as my_int
import Data_quantification_function_parse_bundle as my_pb
import Data_quantification_function_plotting as my_plot
# import Data_quantification_functions_plotPolar4 as my_polar

In [9]:
"""============== main =============="""

'============== main =============='

In [10]:
sns.set_style("dark")

In [11]:
"""Load data"""
### load summaries
summary_df = pd.read_csv(os.path.join(dataPrefix, summaryFolder, summaryName))
image_list = summary_df.loc[:,'Image_Name'].unique()
ROI_list = summary_df.loc[:,'ROI_Name'].unique()
isExtendedTargetList = False

In [12]:
### 3 or 4 color staining
if(isExtendedTargetList == True):
    targetIndexMatch = {0:0, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:30, 8:20, 9:50, 10:40}
    targetIndexMatch_rev = {0:0, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 30:7, 20:8, 50:9, 40:10}
    ColorCode = {0:'#FFFFFF', 1:'#52FEFE', 2:'#1FF509', 3: '#FF0000', 4: '#CFCF1C', 5: '#FF00FF', 6: '#FFAE01', 20:'#1FF509', 30:'#FF0000', 40:'#CFCF1C', 50:'#FF00FF'}
    channel_mapping = {'RFP':0, 'GFP':1, 'R3_1':2, 'R4_1':3, 'R3_2':4, 'R4_2':5, 'FasII': 6, '24B10': 7, 0:'RFP', 1:'GFP', 2:'R3_1', 3:'R4_1', 4:'R3_2', 5:'R4_2', 6:'FasII', 7:'24B10'}
    matching_info = (targetIndexMatch, ColorCode, channel_mapping, targetIndexMatch_rev)
else:
    targetIndexMatch = {0:0, 1:2, 2:3, 3:4, 4:5, 5:7}
    targetIndexMatch_rev = {0:0, 2:1, 3:2, 4:3, 5:4, 7:5}
    ColorCode = {1:'#00FFFF', 2:'#1FF509', 3: '#FF0000', 4: '#CFCF1C', 5: '#FF00FF', 6: '#FFAE01', 7:'#983535', 0:'#FFFFFF'}
    channel_mapping = {'RFP':0, 'GFP':1, 'R3_1':2, 'R4_1':3, 'R3_2':4, 'R4_2':5, 'R3_3': 6, 0:'RFP', 1:'GFP', 2:'R3_1', 3:'R4_1', 4:'R3_2', 5:'R4_2', 6:'R3_3'}
    channel_cmap = {0:'Reds', 1: 'Greens', 2:'Reds', 3: 'Greens', 4:'Reds', 5: 'Greens', 6:'Reds'}
    matching_info = (targetIndexMatch, ColorCode, channel_mapping, channel_cmap, targetIndexMatch_rev)

In [13]:
i_image = 0

In [14]:
### load other data
imageName = image_list[i_image]
ROIName = ROI_list[i_image]
ROI_df = pd.read_csv(os.path.join(dataPrefix, ROIFolder, ROIName))
ROI_df.rename(columns = {' ':'No'}, inplace = True)
annot_df = summary_df.groupby(['Image_Name']).get_group(image_list[i_image]).reset_index(drop = True)


In [15]:
%time image = img_as_float(skiIo.imread(os.path.join(dataPrefix, imageFolder, imageName)))
nChannels = min(image.shape)
if(nChannels == 2):
    image_shape = (image.shape[0], image.shape[2], image.shape[3])
else:
    image_shape = (image.shape[0], image.shape[1], image.shape[2])
M2P_ratio = (summary_df.iloc[0]['imgX_pixel']/summary_df.iloc[0]['imgX_um'], summary_df.iloc[0]['imgY_pixel']/summary_df.iloc[0]['imgY_um'])

CPU times: user 2.02 s, sys: 3.29 s, total: 5.3 s
Wall time: 6.99 s


In [16]:
image.shape

(190, 2, 685, 2140)

In [18]:
min(image.shape)

2

In [19]:
""" Process annotation info"""
bundles_df = my_pb.getBundlesInfo(ROI_df, annot_df, M2P_ratio[0], M2P_ratio[1], isExtendedTargetList)
annot_bundles_df = bundles_df.dropna(axis=0, how='any', inplace = False)
annot_bundles_df_good = my_pb.Good_QC_df(annot_bundles_df)
annot_bundles_df_bad = my_pb.Bad_QC_df(annot_bundles_df)

/Users/lily/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


---df_annot---
1
7
13
19
25
32
33
12
41
17
23
47
49


In [ ]:
""" Process images """
### number of channels
nChannels = min(image.shape)# number of channels of original image
num_norm_channels = len(channel_cmap.keys()) # number of channels of normalized image

if(nChannels == 2):
    print("2 channels!")
    ### normalize channels
    image_norm = np.empty(image_shape + (num_norm_channels,), dtype=image[:,0,:,:].dtype, order='C')
    thr = np.zeros((2))
    
    # RFP_norm
    %time image_norm[:,:,:,0] = exposure.rescale_intensity(image[:,0,:,:], in_range = 'image', out_range='dtype')
    # GFP_norm
    %time image_norm[:,:,:,1] = exposure.rescale_intensity(image[:,1,:,:], in_range = 'image', out_range='dtype')    
    
    del image
    
    print("gfp threshold!")
    %time thr[0] = filters.threshold_isodata(image_norm[:,:,:,1])
    %time thr[1] = filters.threshold_mean(image_norm[:,:,:,1])

    print("histogram matching!")
    %time gfp = transform.match_histograms(image_norm[:,:,:,1], image_norm[:,:,:,0])
    R3 = image_norm[:,:,:,0] - gfp*1.5
    R3[R3<0] = 0
    %time image_norm[:,:,:,2] = exposure.rescale_intensity(R3, in_range = 'image', out_range='dtype')
    R4 = image_norm[:,:,:,0] * gfp
    %time image_norm[:,:,:,3] = exposure.rescale_intensity(R4, in_range = 'image', out_range='dtype')
    
    gfp_thr = morphology.binary_opening((image_norm[:,:,:,1]>thr[0])*1)
    %time image_norm[:,:,:,4] = exposure.rescale_intensity(image_norm[:,:,:,0] * (1-gfp_thr), in_range = 'image', out_range='dtype')
    %time image_norm[:,:,:,5] = exposure.rescale_intensity(morphology.closing(image_norm[:,:,:,1]*((image_norm[:,:,:,1]>((thr[0] + thr[1])/2))*1)))

    del R3, R4, gfp, gfp_thr
    
if(nChannels == 3):
    print("3 channels!")
    ### Seperate channels
    GFP = image[:,:,:,1]
    RFP = image[:,:,:,0]
    Cy5 = image[:,:,:,2]

    ### normalize GFP & RFP channel
    image_norm = np.empty(image_shape + (num_norm_channels,), dtype=GFP.dtype, order='C')
    
#     %time GFP_norm = exposure.equalize_adapthist(GFP)
    %time GFP_norm = exposure.rescale_intensity(GFP, in_range = 'image', out_range='dtype')
    
#     %time RFP_norm = exposure.equalize_adapthist(RFP)
    %time RFP_norm = exposure.rescale_intensity(RFP, in_range = 'image', out_range='dtype')
    
#     %time Cy5_norm = exposure.equalize_adapthist(GFP)
    %time Cy5_norm = exposure.rescale_intensity(Cy5, in_range = 'image', out_range='dtype')
    
    image_norm[:,:,:,0] = RFP_norm
    image_norm[:,:,:,1] = GFP_norm
    image_norm[:,:,:,4] = Cy5_norm

    R3 = RFP_norm - GFP_norm
    R3[R3<0] = 0
#     %time R3_norm = exposure.equalize_adapthist(R3)
    %time R3_norm = exposure.rescale_intensity(R3, in_range = 'image', out_range='dtype')
    image_norm[:,:,:,2] = R3_norm

    R4 = RFP_norm * GFP_norm
#     %time R4_norm = exposure.equalize_adapthist(R4)
    %time R4_norm = exposure.rescale_intensity(R4, in_range = 'image', out_range='dtype')
    image_norm[:,:,:,3] = R4_norm
    
elif(nChannels == 4):
    print("4 channels!")
    ### normalize channels
    image_norm = np.empty(image_shape + (num_norm_channels,), dtype=image[:,:,:,1].dtype, order='C')
    thr = np.zeros((2))
    
    # RFP_norm
    %time image_norm[:,:,:,0] = exposure.rescale_intensity(image[:,0,:,:], in_range = 'image', out_range='dtype')
    # GFP_norm
    %time image_norm[:,:,:,1] = exposure.rescale_intensity(image[:,1,:,:], in_range = 'image', out_range='dtype')    
    
    print("gfp threshold!")
    %time thr[0] = filters.threshold_isodata(image_norm[:,:,:,1])
    %time thr[1] = filters.threshold_mean(image_norm[:,:,:,1])

    print("histogram matching!")
    %time gfp = transform.match_histograms(image_norm[:,:,:,1], image_norm[:,:,:,0])
    
    print("R3/R4 v1")
    R3 = image_norm[:,:,:,0] - gfp
    R3[R3<0] = 0
    %time image_norm[:,:,:,2] = exposure.rescale_intensity(R3, in_range = 'image', out_range='dtype')
    R4 = image_norm[:,:,:,0] * gfp
    %time image_norm[:,:,:,3] = exposure.rescale_intensity(R4, in_range = 'image', out_range='dtype')
    
    print("R3/R4 v2")
    gfp_thr = morphology.binary_opening((image_norm[:,:,:,1]>thr[0])*1)
    %time image_norm[:,:,:,4] = exposure.rescale_intensity(image_norm[:,:,:,0] * (1-gfp_thr), in_range = 'image', out_range='dtype')
    %time image_norm[:,:,:,5] = exposure.rescale_intensity(morphology.closing(image_norm[:,:,:,1]*((image_norm[:,:,:,1]>((thr[0] + thr[1])/2))*1)))
    
    print("R3 v3")
    R3 = image_norm[:,:,:,0] - gfp*5
    R3[R3<0] = 0
    %time image_norm[:,:,:,6] = exposure.rescale_intensity(R3, in_range = 'image', out_range='dtype')

    del R3, R4, gfp, gfp_thr

2 channels!
CPU times: user 10.5 s, sys: 20 s, total: 30.5 s
Wall time: 45.8 s
CPU times: user 11.7 s, sys: 36.4 s, total: 48.1 s
Wall time: 1min 12s
gfp threshold!
CPU times: user 11.1 s, sys: 25.9 s, total: 37 s
Wall time: 47.1 s
CPU times: user 3.78 s, sys: 17 s, total: 20.7 s
Wall time: 23.7 s
histogram matching!


In [ ]:
""" Plot individual bundles """

In [ ]:
# for ind in range(len(annot_bundles_df.index)):
ind = 6
bundle_No = list(annot_bundles_df.index)[ind]
plotSettings = False, False, False, True #isPlotR3Line, isPlotR4Line, isPlotR4s, isLabelOff
# image_norm[:,:,:,2] = filter_test_rfp[:,:,:,2]
# image_norm[:,:,:,3] = filter_test_gfp[:,:,:,2]
# my_plot.plotIndividualBundles(bundle_No, bundles_df, image_norm[:,:,:,0:6], M2P_ratio[0], M2P_ratio[1], z_offset, plotSettings, matching_info)
my_plot.plotIndividualBundles(bundle_No, bundles_df, image_norm[:,:,:,0:6], M2P_ratio[0], M2P_ratio[1], z_offset, plotSettings, matching_info)
plt.show()

In [ ]:
""" get intensity matrix for good bundles"""

In [ ]:
### initialization
print('-----' + image_list[i_image] + '------')
matrixY = num_angleSection + 2*num_outsideAngle + 1
matrixX = num_Xsection + 1
matrixZ = z_offset * 2 + 1
IntensityMatrix = np.zeros((len(annot_bundles_df.index), 2,2,6, matrixY, matrixX, matrixZ))
IntensityMatrix = IntensityMatrix - 100
#IntensityMatrix.shape = ind, slicetype, centertype, channelNo, matrixX, matrixY, matrixZ
params = [];
#params.shape = ind, slicetype, centertype
rel_points = np.zeros((len(annot_bundles_df.index), 2, 2, 9))
num_norm_channels = len(channel_cmap.keys())
#params.shape = ind, slicetype, centertype

In [ ]:
IntensityMatrix.shape

In [ ]:
### parse bundles
# for ind in range(len(annot_bundles_df.index)):
for ind in [0]:
    bundle_No = list(annot_bundles_df.index)[ind]
    R_Z = int(bundles_df.loc[bundle_No,'coord_Z_R' + str(3)]) - 1
    print("Bundle No: ", bundle_No)
    #     DV = bundles_df.loc[bundle_No, 'Orientation_DV']
    #     AP = bundles_df.loc[bundle_No, 'Orientation_AP']

    ### targets info
    indTs, coordTs = my_help.getTargetCoords(bundle_No, bundles_df, targetIndexMatch)
    coord_Center = my_help.getBundleCenter(bundle_No, bundles_df)
    coordR4s = my_help.getRxCoords(bundle_No, bundles_df, indTs, 4)
    coordR3s = my_help.getRxCoords(bundle_No, bundles_df, indTs, 3)
    coordRs = np.concatenate((coordR4s, coordR3s))

    ### slice info
    SliceZeroPoint = coordTs[targetIndexMatch_rev[7],:] # T3'
    SliceOnePoint = coordTs[targetIndexMatch_rev[3],:] # T3

    ## slice radius info
    LengthOnePoint = coordTs[targetIndexMatch_rev[4],:]

    CenterPoints = [coordTs[0,:], coord_Center[0,:]]

    Rcell_nums = [4,4]

    printingParams = [False, False]

    ### get slicing params, calculate matrix
    pp = [[], []]
    for centertype in [0,1]:
        analysis_params = [analysisParams[0], analysisParams[1], analysisParams[2], analysisParams[3], analysisParams[4][centertype]]
        print(centertype)
        bundleParams = [bundle_No, indTs, coordTs, coord_Center, SliceZeroPoint, SliceOnePoint, LengthOnePoint, CenterPoints[centertype], Rcell_nums[centertype]]
        for slicetype in [0,1]:
            ## get slicing params
            if(slicetype == 0):
                pp_i, rel_points_i = my_int.getSliceParams_v1(analysis_params, bundles_df, bundleParams, printingParams, matching_info[4])
            elif(slicetype == 1):
                pp_i, rel_points_i = my_int.getSliceParams_v3(analysis_params, bundles_df, bundleParams, printingParams, matching_info[4])
            pp[slicetype].append(pp_i)
            rel_points[slicetype, centertype, ind, :] = rel_points_i
            
            # calculate matrix
            for channelNo in range(num_norm_channels):
#             for channelNo in [0]:
                %time IntensityMatrix[ind, slicetype, centertype, channelNo,:,:,:] = my_int.getIntensityMatrix_new(pp_i, image_norm, channel_mapping[channelNo], channel_mapping)
    params.append(pp)
            


In [ ]:
import Data_quantification_function_plotting15 as my_plot

In [ ]:
### plotting
for ind in [0]:
    bundle_No = list(annot_bundles_df.index)[ind]
    R_Z = int(bundles_df.loc[bundle_No,'coord_Z_R' + str(3)]) - 1
    print("Bundle No: ", bundle_No)
    #     DV = bundles_df.loc[bundle_No, 'Orientation_DV']
    #     AP = bundles_df.loc[bundle_No, 'Orientation_AP']

    ### targets info
    indTs, coordTs = my_help.getTargetCoords(bundle_No, bundles_df, targetIndexMatch)
    coord_Center = my_help.getBundleCenter(bundle_No, bundles_df)
    coordR4s = my_help.getRxCoords(bundle_No, bundles_df, indTs, 4)
    coordR3s = my_help.getRxCoords(bundle_No, bundles_df, indTs, 3)
    coordRs = np.concatenate((coordR4s, coordR3s))

    ### plotting heatmap
    for centertype in [0]:
        for slicetype in [0]:
            pp_i = params[0][slicetype][centertype]
            rel_points_i = rel_points[slicetype, centertype, ind, :]
            
            matrix = my_help.delete_zero_columns(IntensityMatrix[ind, centertype, slicetype, :, :, :, :], -100, 3)
#             matrix = IntensityMatrix[ind, centertype, slicetype, :, :, :, :]
    
            plt.ioff()
            img_name = image_list[i_image]
            ori_X = np.round(np.linspace(0, radiusExpanseRatio[centertype], matrix.shape[2]), 2)
            tickParams = [2, 1, ori_X, 21] ### tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y
            figParams = [centertype, pp_i, figOutPrefix, img_name, slicetype, radiusExpanseRatio[centertype]]
            for thrFunction in [0]: # different thresholding methods
                if(thrFunction == 0):
                    thrs = np.zeros((6))
                elif(thrFunction == 1):
                    thrs = thr_otsu
                #     elif(thrFunction == 2):
                #         thrs = thr_li
                plotOptions = [True, True, True, True, True, thrs, thrFunction] ### isPlotLine, isLabelOff, isSave, isTrueXTick, isOriTick, thrFunction            
                fig = my_plot.plotBundleVsMatrix_all(bundle_No, bundles_df, image_norm, matrix, figParams, tickParams, plotOptions, matching_info)
#                 plt.close(fig)
                plt.show()
            ## plotting: polar
            figParams = pp_i, figOutPrefix, img_name, radiusExpanseRatio[centertype], centertype, slicetype
            plotOptions = [True, True] # isLabelOff, isSave
            for channelNo in range(num_norm_channels):
                analysis_params = [analysisParams[0], analysisParams[1], analysisParams[2], analysisParams[3], analysisParams[4][centertype]]
                fig = my_plot.plotPolar(bundle_No, bundles_df, image_norm, analysis_params, channelNo, matrix, figParams, plotOptions, matching_info, rel_points[slicetype, centertype, ind, :])
        #         plt.close(fig)
                plt.show()

In [ ]:
image_norm

In [ ]:
### Save matrixes and parameters

In [ ]:
categoryID = annot_bundles_df.iloc[0]['CategoryID']
sampleID = annot_bundles_df.iloc[0]['SampleID']
regionID = annot_bundles_df.iloc[0]['RegionID']

In [ ]:
    ### plotting: heatmap
#     matrix = IntensityMatrix[:, ind, :,:,:]
#     plt.ioff()
#     img_name = image_list[i_image]
#     ori_X = np.round(np.linspace(0, radiusExpanseRatio, matrix.shape[2]), 2)
#     tickParams = [2, 1, ori_X, 21] ### tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y
#     figParams = [mind, pp[0:7], figOutPrefix, img_name, sliceTypeNo, radiusExpanseRatio]
#     for thrFunction in [0]: # different thresholding methods
#         if(thrFunction == 0):
#             thrs = np.zeros((6))
#         elif(thrFunction == 1):
#             thrs = thr_otsu
#         #     elif(thrFunction == 2):
#         #         thrs = thr_li
#         plotOptions = [True, True, True, True, False, thrs, thrFunction] ### isPlotLine, isLabelOff, isSave, isTrueXTick, isOriTick, thrFunction            
#         fig = my_plot.plotBundleVsMatrix_all(bundle_No, bundles_df, image_norm, matrix, figParams, tickParams, plotOptions, matching_info)
#         plt.close(fig)

#     ### plotting: polar
#     figParams = pp[0:7], figOutPrefix, img_name, radiusExpanseRatio
#     plotOptions = [True, True] # isLabelOff, isSave
#     for channelNo in range(6):
#         fig = my_plot.plotPolar(bundle_No, bundles_df, image_norm, analysisParams, channelNo, matrix, figParams, plotOptions, matching_info)
# #         plt.close(fig)


In [ ]:
outputData = {}
outputData['categoryID'] = categoryID
outputData['sampleID'] = sampleID
outputData['regionID'] = regionID
outputData['IntensityMatrix'] = IntensityMatrix
outputData['Parameter'] = params

In [ ]:
import datetime

In [ ]:
now = datetime.datetime.now()
date_info = str(now.year)+str(now.month)+str(now.day)
outputname = categoryID + '_sample' + str(sampleID) + '_region' + str(regionID) + '_v' + date_info + '.pickle'

In [ ]:
outputDir = os.path.join(dataOutPrefix)
my_help.check_dir(outputDir)
outputDir = os.path.join(outputDir,categoryID)
my_help.check_dir(outputDir)
# outputname = categoryID + '_sample' + str(sampleID) + '_region' + str(regionID) +  + '.pickle'
outputname = os.path.join(outputDir, outputname)
pickle_out = open(outputname,"wb")
pickle.dump(outputData, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open(outputname,"rb")
outputData[i] = pickle.load(pickle_in)

In [ ]:
# ind = 1

# # mind = 2
# # sliceType = 1
# channelNo = 2
# # 
# channel = channel_mapping[channelNo]
# bundle_No = list(annot_bundles_df.index)[ind]

# # plt.ioff()
# img_name = image_list[i_image]
# figParams1 = [channel, mind, pp[sliceTypeNo][mind], figOutPrefix, img_name, sliceTypeNo, radiusExpanseRatio]
# figParams2 = [mind, pp[sliceTypeNo][mind], figOutPrefix, img_name, sliceTypeNo, radiusExpanseRatio]

# tickParams = [1, 2, 5, [-1, -0.5, 0, 0.5, 1]] ### tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y
# plotOptions = [True, True, False, True, False] ### isPlotLine, isLabelOff, isSave, isTrueXTick, isMask
# matrix1 = IntensityMatrix[sliceTypeNo, channelNo, mind, ind, :,:,:]
# matrix2 = IntensityMatrix[sliceTypeNo, :, mind, ind, :,:,:]
# # fig1 = my_plot.plotBundleVsMatrix(bundle_No, bundles_df, image_norm, matrix1, figParams1, tickParams, plotOptions, matching_info)
# fig2 = plotBundleVsMatrix_all(bundle_No, bundles_df, image_norm, matrix2, figParams2, tickParams, plotOptions, matching_info)
# # plt.close(fig)